# Analisi statistica intervalli di ricorrenza extreme returns

Questo notebook contiene le analisi statistiche fatte per il paper.

Il flusso è il seguente:

- [x] utilizzo del dataset *Dow Jones* con la massima ampiezza storica disponibile (1985 - 2019)
- [x] calcolo dei log returns
- [x] plot degli intervalli di ricorrenza per il Dow Jones, con metodi diversi:
    - [x] quantile threshold al
        - [x] 95%
        - [x] 97.5%
        - [x] 99%
        - [x] verifica della relazione $\tau_Q = \frac{Q}{1 - Q}$ dove $Q$ è il quantile scelto (0.95, 0.975, 0.99), $\tau_Q$ l'intervallo di ricorrenza medio, e confronto con l'evidenza dei dati
    - [x] peak-over-threshold definito come $pot = \mu \pm m \cdot \sigma$: ricavare analiticamente m come $m = \frac{q_x - \mu}{\sigma}$ dove $q_x$ è il quantile di ordine $x$
        
Infine, tutto dovrà essere rifatto per una azione dell'*S&P500*, non per il Dow Jones come di seguito.

In [ ]:
import os
import time
import datetime
from typing import List
import itertools
import pickle
import math

import numpy as np
import pandas as pd
import scipy.stats
import scipy.special as sfun
from scipy.stats import genextreme as gev

from statsmodels.tsa import stattools
from statsmodels.graphics import tsaplots

import matplotlib.pyplot as pl
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

from tqdm import tqdm
import ipdb

import numba

# import stimatore di Hill
import tail_estimation
from my_timeit import timeit

In [ ]:
%pdb on

In [ ]:
%load_ext line_profiler
%load_ext autoreload
%load_ext cython

In [ ]:
%autoreload 5

In [ ]:
return_type = ['pos', 'neg', 'abs']
quantile_type = ['95', '97.5', '99', 'evt']
distribution_type = ['weibull', 's-exp', 'q-exp']

colors = {
    'pos': 'seagreen',
    'neg': 'darkred',
    'abs': 'royalblue',
}

legend_labels = {
    'pos': r'$r$',
    'neg': r'$-r$',
    'abs': r'$|r|$',
}

## 1. Caricamento dei dati e divisione train-test

In [ ]:
data_path = "/Users/pietro/Google Drive/OptiRisk Thesis/data"
djia_path = os.path.join(data_path, 'DJIA.csv')

Conversione delle date e settaggio dell'index del dataframe

In [ ]:
djia = pd.read_csv(djia_path)
djia.loc[:, 'Date'] = pd.to_datetime(djia['Date'], format="%Y-%m-%d")
djia.index = djia['Date']
djia.drop(columns=['Date'], inplace=True)
djia.head()

Ora dichiariamo le date in cui bisogna dividere i dati, visto che ci sono state crisi nei mesi/anni successivi:

In [ ]:
split_dates = {
    'insurance': datetime.datetime(1987, 1, 1), # insurance companies crisis
    'dot-com': datetime.datetime(2000, 1, 1), # dot-com bubble explodes
    'subprime-crisis': datetime.datetime(2007, 1, 1), # subprime crisis
    'eu-debt': datetime.datetime(2011, 1, 1), # EU sovereign debt crisis
}

## 2. Calcolo log-returns

In [ ]:
log_returns = np.log(djia.loc[:, ['Adj Close']]).diff(periods=1).iloc[1:, :]
log_returns.head()

In [ ]:
fig, ax = pl.subplots(nrows=1, ncols=2, figsize=(18, 7))

ax[0].plot(djia['Adj Close'])
ax[0].set(xlabel='Date', ylabel='DJIA', title='Dow Jones index value')
sns.despine()

ax[1].plot(log_returns, label='Log Returns')
ax[1].set(xlabel='Date', ylabel='Log Returns', title='Dow Jones Log Returns')
sns.despine()

Creo una funzione per dividere il dataset prima e dopo gli eventi critici:

In [ ]:
def divide(data: pd.DataFrame, before_date: datetime.datetime):
    """Split the data before and after the before_date."""
    before = data[data.index < before_date]
    after = data[data.index >= before_date]
    
    return before, after

In [ ]:
returns_before_after = {
    event: divide(log_returns, split_dates[event])
    for event in split_dates.keys()
}

Ora scelgo quale sia la data di splitting e faccio le analisi con quella:

In [ ]:
split_key = 'subprime-crisis'

## 3. Stima dei parametri della distribuzione GEV

In questa sezione si replica la sezione 4.1 del paper.

Secondo la [Extreme Value Theory](https://en.wikipedia.org/wiki/Extreme_value_theory), trovare gli estremi significa trovare un gruppo di dati $x \geq x_t$ dove $x_t$ è l'*extreme value threshold*, e che soddisfi la GEV ([Generalized Extreme Values Distribution](https://en.wikipedia.org/wiki/Generalized_extreme_value_distribution)) che ha distribuzione cumulativa:

\begin{align*}
    G(x) &= exp\left[- \left(1 + \xi \frac{x - \mu}{\sigma}\right)^{-1/\xi}\right] \; for \; \xi \neq 0\\ 
    G(x) &= exp\left[-exp\left(\frac{x - \mu}{\sigma}\right)\right] \; for \; \xi = 0\\ 
\end{align*}

dove $\xi$ è lo *shape parameter* che determina la forma della coda, $1/\xi$ è il *tail exponent* della distribuzione.

Per trovare il threshold $x_t$, usiamo questo metodo:

1. sort dei dati (tutti i log-returns) in ordine discendente (o non-ascendente) per avere la sequenza $x_1 \geq x_2 \geq \ldots \geq x_n$
2. applicare lo [stimatore di Hill](https://en.wikipedia.org/wiki/Heavy-tailed_distribution#Hill's_tail-index_estimator) dove $n$ è il numero di samples, $k$ l'indice del k-esimo dato più grande (posizione k nella sequenza ordinata) chiamato *k-th order statistic*

$$\hat{\xi}_{k,n} = \frac{1}{\gamma - 1} = \frac{1}{k}\sum_{i=1}^{k}log\left(\frac{x_i}{x_{k+1}}\right)$$

3. calcolare la statistica di [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test#Kolmogorov%E2%80%93Smirnov_statistic) per quantificare il fitting tra la distribuzione GEV così ricavata (con $\hat{\xi}_{k,n}$ come esponente) e quella empirica della coda, dove la coda è rappresentata dai returns che sono $x \geq x_t$, cioé quelli ordinati discendenti con indice $i < k$
4. scegliere $k$ (e di conseguenza $x_t$ che non è altro che il k-esimo elemento dei return ordinati discendenti) come il valore associato alla *minima* statistica di Kolmogorov-Smirnov

Questo flusso viene applicato a tutti e 3 i tipi di returns considerati: positivi, negativi e assoluti.

Cominciamo:

### 3.1 Stima di $\gamma$

In [ ]:
@timeit
def estimate_shape_param(x: np.ndarray, get_optimum=False, n_times=1):
    """Estimate the shape parameter gamma for the GEV using Hill estimator.
    Input MUST be sorted in descending order.
    """
    # xi_estimation[0]: order statistics
    # xi_estimation[1]: tail index estimates (xi su Wikipedia, gamma nel paper)
    # xi_estimation[2]: optimal order statistics (k)
    # xi_estimation[3]: tail index for the optimal order statistics

    # xi_estimation[4]: array of fractions of order statistics used for the 1st bootstrap sample
    # xi_estimation[5]: corresponding AMSE values
    # xi_estimation[6]: fraction of order statistics corresponding to the minimum of AMSE for the 1st bootstrap
    # xi_estimation[7]: index of the 1st bootstrap sample's order statistics array corresponding to the
    #                   minimization boundary set by eps_stop parameter

    # xi_estimation[8]: array of fractions of order statistics used for the 2nd bootstrap sample
    # xi_estimation[9]: corresponding AMSE values
    # xi_estimation[10]: fraction of order statistics corresponding to the minimum of AMSE for the 2nd bootstrap
    # xi_estimation[11]: index of the 2nd bootstrap sample's order statistics array corresponding to the
    #                   minimization boundary set by eps_stop parameter
    assert isinstance(get_optimum, bool)
    
    # check descending
    assert np.all(np.diff(x) <= 0.0)
    
    xis = np.zeros((n_times, x.shape[0] - 1))
    optimal_kappas = np.zeros((n_times, ), dtype=np.int)
    optimal_xis = np.zeros((n_times, ), dtype=np.float64)
    
    for i in range(n_times):
        xi_estimation = tail_estimation.hill_estimator(x, bootstrap=get_optimum)

        kappas = xi_estimation[0]
        xi_hill = xi_estimation[1]
        
        xis[i, :] = xi_hill[:]
        
        if get_optimum:
            optimal_kappas[i] = xi_estimation[2]
            optimal_xis[i] = xi_estimation[3]
            
    xis = np.mean(xis, axis=0)  # per ogni colonna (k) la media delle xi che ha trovato
    
    if get_optimum:
        optimal_k = int(round(np.mean(optimal_kappas)))
        optimal_xi = np.mean(optimal_xis)
        optimal_gamma = 1.0 + (1.0 / optimal_xi)
        
        return {
            'kappas': kappas,
            'xis': xis,
            'gammas': 1.0 + (1.0 / xis),
            'k_opt': optimal_k,
            'xi_opt': optimal_xi,
            'gamma_opt': optimal_gamma,
        }
    else:
        return {
            'kappas': kappas,
            'xis': xis,
            'gammas': 1.0 + (1.0 / xis),
        }

Calcolo i return ordinati dal più grande al più piccolo per i positivi, negativi e assoluti:

In [ ]:
# 0. scelta di dove fermarsi, qui prima della crisi finanziaria 2007-2008
lr_before, lr_after = returns_before_after[split_key]

# returns contiene il training set
# 1. sorting dei returns - va fatto sia per il positivo, che per il negativo, che per gli assoluti
returns = {
    'pos': lr_before['Adj Close'][lr_before['Adj Close'] > 0.0],
    'neg': lr_before['Adj Close'][lr_before['Adj Close'] < 0.0],
    'abs': lr_before['Adj Close'].abs(),
}

sorted_positive_lr = returns['pos'].sort_values(ascending=False)
sorted_negative_lr = (-returns['neg']).sort_values(ascending=False)
sorted_absolute_lr = returns['abs'].sort_values(ascending=False)

eps = 5e-6

sorted_lr = {
    'pos': sorted_positive_lr[sorted_positive_lr >= eps],
    'neg': sorted_negative_lr[sorted_negative_lr >= eps],
    'abs': sorted_absolute_lr[sorted_absolute_lr >= eps],
}

Calcolo la stima di $\xi = \frac{1}{\gamma - 1}$ con lo stimatore di Hill per i tre returns:

In [ ]:
hill_estimation = {
    ret_type: estimate_shape_param(sorted_lr[ret_type].values, get_optimum=True, n_times=30)
    for ret_type in return_type
}

print("")
title_format = "{:>15}"*4
row_format = "{:>15}" + "{:>15.3f}" * 2 + "{:>15}"
print(title_format.format('Return type', 'Xi', 'gamma = -c', 'k'))
print("-"*60)
for ret_type in return_type:
    est = hill_estimation[ret_type]
    print(row_format.format(ret_type, est['xi_opt'], est['gamma_opt'], est['k_opt']))

### 3.2 Plot di $\xi_{k, n}$ e $\gamma$ al variare di k e dei returns

Plot di $\xi$ e $\gamma$ al variare di $k$, cioè del threshold, e plot di $\xi$ e $\gamma$ al variare dei return, per vedere come cambia a seconda di quale return si prenda come threshold:

In [ ]:
# plot delle stime di xi e gamma al variare di k
truncation = 100

fig, ax = pl.subplots(nrows=2, ncols=2, figsize=(18, 14))

# Xi
for ret_type in return_type:
    est = hill_estimation[ret_type]
    x = est['kappas'][truncation:]
    y = est['xis'][truncation:]

    ax[0][0].plot(x, y, color=colors[ret_type], label=legend_labels[ret_type])
    ax[0][0].plot(est['k_opt'], est['xi_opt'], marker='s', markersize=5, color=colors[ret_type])

ax[0][0].set(
    xlabel=r'$k$',
    ylabel=r'$\hat{\xi}_{k,n} = \frac{1}{\hat{\gamma} - 1}$',
    title=r'Hill estimation of $\hat{\xi}_{k,n}$ as function of $k$'
)
ax[0][0].legend()


# gamma
for ret_type in return_type:
    est = hill_estimation[ret_type]
    x = est['kappas'][truncation:]
    y = est['gammas'][truncation:]

    ax[0][1].plot(x, y, color=colors[ret_type], label=legend_labels[ret_type])
    ax[0][1].plot(est['k_opt'], est['gamma_opt'], marker='s', markersize=5, color=colors[ret_type])

ax[0][1].set(
    xlabel=r'$k$',
    ylabel=r'$\hat{\gamma}_{k,n}$',
    title=r'Hill estimation of $\hat{\gamma}_{k,n}$ as function of $k$'
)
ax[0][1].legend()


# seconda riga, in funzione dei log-returns
# Xi
for ret_type in return_type:
    est = hill_estimation[ret_type]
    x = sorted_lr[ret_type].values[1:][::-1][truncation:]
    y = est['xis'][truncation:]

    ax[1][0].semilogx(x, y, color=colors[ret_type], label=legend_labels[ret_type])

ax[1][0].set(
    xlabel=r'$r$, $-r$, $|r|$',
    ylabel=r'$\hat{\xi}_{k,n} = \frac{1}{\hat{\gamma} - 1}$',
    title=r'Hill estimation of $\hat{\xi}_{k,n}$ as function of the returns'
)
ax[1][0].legend()

# gamma
for ret_type in return_type:
    est = hill_estimation[ret_type]
    x = sorted_lr[ret_type].values[1:][::-1][truncation:]
    y = est['gammas'][truncation:]

    ax[1][1].semilogx(x, y, color=colors[ret_type], label=legend_labels[ret_type])

ax[1][1].set(
    xlabel=r'$r$, $-r$, $|r|$',
    ylabel=r'$\hat{\gamma}_{k,n}$',
    title=r'Hill estimation of $\hat{\gamma}_{k,n}$ as function of sorted returns'
)
ax[1][1].legend()

sns.despine()

### 3.3 Fitting della GEV

Ora biogna passare a fittare la GEV e calcolare la statistica KS per ogni valore di $k$ (e quindi del return che funge da threshold, $x_t$).

Andremo poi a scegliere il valore di $k$ che minimizza la KS.

Creiamo allora una funzione apposita:

In [ ]:
@timeit
@numba.jit(nopython=False, parallel=True, nogil=True)
def fit_gev(lr, xi, k, size=None, n_tries=1):
    """Find the best fitting GEV to the tail distribution of data in lr."""
#     assert isinstance(xi, np.ndarray)
#     assert isinstance(k, np.ndarray)
#     assert xi.shape == k.shape
    
    # check descending
#     assert np.all(np.diff(lr) <= 0.0)
    nk = k.shape[0]
    n_xi = xi.shape[0]
    
    ks = np.zeros((n_xi, ))
    pvals = np.zeros((n_xi, ))
    fits = []
    
    print("Start fitting")
    for i in numba.prange(nk):
        current_k = k[i]
        current_xi = xi[i]
        
        threshold = lr[current_k]
        tail_data = lr[:current_k]
        
        if size:
            ss = size
        else:
            ss = current_k
        
        fit = gev.fit(tail_data, fix_c=-current_xi)
        fits.append(fit)
        
        k_stat_temp = np.zeros((n_tries, ))
        pval_temp = np.zeros((n_tries, ))
        
        c = fit[0]
        loc = fit[1]
        scale = fit[2]
        
        for j in range(n_tries):
            rvs = gev.rvs(c, loc, scale, ss)

            kk, pv = scipy.stats.ks_2samp(tail_data, rvs)
            k_stat_temp[j] = kk
            pval_temp[j] = pv
        
        ks[i] = np.mean(k_stat_temp)
        pvals[i] = np.mean(pval_temp)
        
    print("Fitting done")
    
    return {
        'ks': ks,
        'p': pvals,
        'fits': fits,
    }

ed applichiamola:

In [ ]:
load = True
npz_filename = f"ks_stat-pvals_{split_key}.npz"
pickle_filename = f"./fits_{split_key}.pickle"

if load:  # just load the already computed KS and p-values
    loaded = np.load(npz_filename)
    
    with open(pickle_filename, 'rb') as infile:
        fits = pickle.load(infile)
    
    kolmog_smirn = {
        'pos': {
            'ks': loaded['ks_pos'],
            'p': loaded['pvals_pos'],
            'fits': fits['pos'],
        },
        'neg': {
            'ks': loaded['ks_neg'],
            'p': loaded['pvals_neg'],
            'fits': fits['neg'],
        },
        'abs': {
            'ks': loaded['ks_abs'],
            'p': loaded['pvals_abs'],
            'fits': fits['abs'],
        }
    }
    
    
else:  # compute them and save them
    size = 10000
    n_tries = 10
    print("\nFitting returns")

    kolmog_smirn = {
        ret_type: fit_gev(
            sorted_lr[ret_type].values,
            hill_estimation[ret_type]['xis'],
            hill_estimation[ret_type]['kappas'],
            size=size,
            n_tries=n_tries,
        )
        for ret_type in return_type
    }
    
    np.savez_compressed(npz_filename, **{
        'ks_pos': kolmog_smirn['pos']['ks'],
        'pvals_pos': kolmog_smirn['pos']['p'],
        'ks_neg': kolmog_smirn['neg']['ks'],
        'pvals_neg': kolmog_smirn['neg']['p'],
        'ks_abs': kolmog_smirn['abs']['ks'],
        'pvals_abs': kolmog_smirn['abs']['p'],
    })
    
    fits = {
        ret_type: kolmog_smirn[ret_type]['fits']
        for ret_type in return_type
    }
    
    with open(pickle_filename, 'wb') as outfile:
        pickle.dump(fits, outfile)

Ora che ho la statistica KS per ogni valore di k e del return, plot delle statistiche rispetto ai returns e a $k$.

In [ ]:
min_pval = 0.05

valid_pvals = {
    ret_type: kolmog_smirn[ret_type]['p'] <= min_pval
    for ret_type in return_type
}

Le figure seguenti mostrano l'andamento di $d_{KS}$ in funzione di k e dei returns ordinati, su scala $x$ semilogaritmica.

In [ ]:
# plot della statistica KS al variare del sorted return e di k
truncation = 100
end = -170
p_labels = {
    'pos': r'valid $p$ for $r$',
    'neg': r'valid $p$ for $-r$',
    'abs': r'valid $p$ for $|r|$',
}

p_height = {
    'pos': 0.0,
    'neg': -0.01,
    'abs': -0.02
}

fig, ax = pl.subplots(nrows=1, ncols=2, figsize=(18, 7))

# d_KS in funzione dei returns
for ret_type in return_type:
    x = sorted_lr[ret_type].values[1:][::-1][truncation:end]
    y = kolmog_smirn[ret_type]['ks'][truncation:end]
    mask = valid_pvals[ret_type]
    x_ok = x[mask[::-1][truncation:end]]
    
    i_min = np.argmin(y)
    
    ax[0].semilogx(
        x,
        y,
        color=colors[ret_type],
        label=legend_labels[ret_type])
    
    ax[0].semilogx(
        x_ok,
        p_height[ret_type] * np.ones((len(x_ok, ))),
        color=colors[ret_type],
        marker='.',
        linestyle='',
        label=p_labels[ret_type]
    )
    
    ax[0].axvline(
        x[i_min],
        linestyle='-.',
        color=colors[ret_type],
        alpha=0.7
    )

ax[0].set(
    xlabel=r'$r$, $-r$, $|r|$',
    ylabel=r'$d_{KS}$',
    title=r'KS statistics as function of the sorted returns'
)
ax[0].legend()


# d_KS in funzione di k
for ret_type in return_type:
    x = hill_estimation[ret_type]['kappas'][truncation:end]
    y = kolmog_smirn[ret_type]['ks'][truncation:end]
    mask = valid_pvals[ret_type]
    k_ok = x[mask[::-1][truncation:end]]
    
    i_min = np.argmin(y)
    
    ax[1].semilogx(
        x,
        y,
        color=colors[ret_type],
        label=legend_labels[ret_type]
    )
    
    ax[1].semilogx(
        k_ok,
        p_height[ret_type] * np.ones((len(k_ok, ))),
        color=colors[ret_type],
        marker='.',
        linestyle='',
        label=p_labels[ret_type]
    )
    
    ax[1].axvline(
        x[i_min],
        linestyle='-.',
        color=colors[ret_type],
        alpha=0.7
    )

ax[1].set(
    xlabel=r'$k$',
    ylabel=r'$d_{KS}$',
    title=r'KS statistics as function of $k$'
)
ax[1].legend()

sns.despine()

Le linee sottostanti le figure, composte in realtà da punti, identificano quei valori per cui il test di Kolmogorov-Smirnov ha dato un _p_-value $p \leq 0.05$, ed è quindi ritenuto statisticamente valido. Si nota come agli estremi di $k$ e dei returns il *p*-value non sia significativo e ci siano grosse oscillazioni, probabilmente dovute a instabilità numeriche nel calcolo della maximum likelihood.

### 3.4 Minima $d_{KS}$ per trovare il miglior fit della GEV

Ottimo, ora bisogna selezionare il minimo valore della statistica KS $d_{KS}$ che abbia un p-value valido ($p < 0.05$).

In [ ]:
def find_min_dks(d_ks, pvals, min_pval=min_pval):
    dks = d_ks.copy()
    invalid = pvals > min_pval
    
    dks[invalid] = np.min(dks) + 1
    i_min = np.argmin(dks)
    
    return dks[i_min], i_min

In [ ]:
min_kolmog_smirn = {
    ret_type: find_min_dks(kolmog_smirn[ret_type]['ks'], kolmog_smirn[ret_type]['p'])
    for ret_type in return_type
}

threshold_evt = {
    'pos': sorted_lr['pos'][min_kolmog_smirn['pos'][1]],
    'neg': sorted_lr['neg'][min_kolmog_smirn['neg'][1]],
    'abs': sorted_lr['abs'][min_kolmog_smirn['abs'][1]],
}

print("")
title_format = "{:>15}"*4
row_format = "{:>15}{:>15.4f}{:>15}{:>15.6f}"
print(title_format.format('Return type', 'Min d_KS', 'i', 'Return'))
print("-"*60)
for ret_type in return_type:
    print(row_format.format(
        ret_type,
        min_kolmog_smirn[ret_type][0],
        min_kolmog_smirn[ret_type][1],
        threshold_evt[ret_type]
    ))

Vediamo graficamente se le distribuzioni fittano bene i dati:

In [ ]:
# preparo i dati per plottare
titles = {
    'pos': 'Positive extreme returns',
    'neg': 'Negative extreme returns',
    'abs': 'Absolute extreme returns',
}

xlabels = {
    'pos': r'Positive extreme returns $r$',
    'neg': r'Negative extreme returns $-r$',
    'abs': r'Absolute extreme returns  $|r|$',
}

labels = {
    'pos': r'$r$',
    'neg': r'$-r$',
    'abs': r'$|r|$',
}

# plot
fig, ax = pl.subplots(nrows=3, ncols=1, figsize=(12, 15), sharex=True)

for ret_type, a in zip(return_type, ax):
    data = sorted_lr[ret_type].values[:min_kolmog_smirn[ret_type][1]]
    sns.distplot(
        data,
        color=colors[ret_type],
        label=legend_labels[ret_type],
        kde=False,
        norm_hist=True,
        ax=a
    )
    
    a.set_xlabel(xlabels[ret_type])

for a, ret_type in zip(ax, return_type):
    i_min = min_kolmog_smirn[ret_type][1]
    best_fit = fits[ret_type][i_min]
    _, b = a.xaxis.get_data_interval()
    x = np.linspace(0, b, 1000)
    pdf = gev.pdf(x, *best_fit)
    a.plot(x, pdf, color=colors[ret_type], label='GEV pdf')
    a.legend()

sns.despine()

Ottimo, sembra che fittino piuttosto bene. A questo punto é finita la sezione 4.1 del paper.

Finora abbiamo quindi ottenuto:

- le distribuzioni di probabilità degli extreme returns (positivi, negativi, assoluti)
- i threshold che massimizzano il fitting della distribuzione *GEV* sugli extreme returns. Tali threshold possono essere quindi usati per determinare quali movimenti siano estremi e quali no!

Concludiamo quindi confrontando i threshold così ottenuti con i threshold del 95% percentile che abbiamo utilizzato finora per le azioni S&P500.

In [ ]:
perc = returns_before_after[split_key][0]['Adj Close'].quantile(q=[0.05, 0.95])

fig, ax = pl.subplots(nrows=1, ncols=1, figsize=(18, 9))

dates = log_returns.index
# training set
ax.plot(
    returns_before_after[split_key][0]['Adj Close'],
    label='Log Returns - train',
    color='slategrey',
    alpha=0.25,
    linestyle='',
    marker='.'
)
# testing set
ax.plot(
    returns_before_after[split_key][1]['Adj Close'],
    label='Log Returns - test',
    color='navy',
    alpha=0.25,
    linestyle='',
    marker='.'
)

ax.plot(dates, perc[0.05] * np.ones(len(dates)), color=colors['neg'], linestyle=':', label='5% percentile')
ax.plot(dates, perc[0.95] * np.ones(len(dates)), color=colors['pos'], linestyle=':', label='95% percentile')
ax.plot(dates, -threshold_evt['neg'] * np.ones(len(dates)), color=colors['neg'], linestyle='--', label=r'$-x_t$ da EVT')
ax.plot(dates, threshold_evt['pos'] * np.ones(len(dates)), color=colors['pos'], linestyle='--', label=r'$x_t$ da EVT')

ax.set(xlabel='Date', ylabel='Log Returns', title='Dow Jones Log Returns and thresholds', ylim=[-0.12, 0.12])
ax.legend(loc='lower right')

sns.despine()

Come si può vedere dal plot, i percentili sono molto più stringenti rispetto al valore che massimizza il fitting della *GEV*. Vediamo anche di confermarlo con un po' di numeri:

In [ ]:
def get_percent(data, thresh_low, thresh_up):
    indexes = np.logical_or(data.values <= thresh_low, data.values >= thresh_up)
    num = len(data[indexes])
    denom = len(data)
    
    return num / denom

extreme_percent_with_percentiles = get_percent(log_returns['Adj Close'], perc[0.05], perc[0.95])
extreme_percent_with_gev = get_percent(log_returns['Adj Close'], -threshold_evt['neg'], threshold_evt['pos'])

print("{:>20}{:>15}".format('Threshold type', 'Extremes %'))
print("-"*35)
print("{:>20}{:>15.3f}".format('percentile 5-95 %', extreme_percent_with_percentiles))
print("{:>20}{:>15.3f}".format('EVT', extreme_percent_with_gev))

In pratica, vuol dire che se usassimo i valori di threshold ricavati dalla *EVT* avremmo un dataset sicuramente più bilanciato, ma c'è da chiedersi se si possano effettivamente allora considerare "estremi". Non è troppo "inclusivo" un tale threshold?

### 3.5 Calcolo dei $\tau_Q$ e delle $Q$

Calcoliamoli per poi usarli nella maximum likelihood estimation.

In [ ]:
# creo i tau e calcolo i Q
tau_q_95 = 1.0 / (1.0 - 0.95)
tau_q_975 = 1.0 / (1.0 - 0.975)
tau_q_99 = 1.0 / (1.0 - 0.99)

# calcolo i quantili equivalenti ai threshold della EVT
Q = {
    ret_type: 1.0 - (sum(returns[ret_type].abs() >= abs(threshold_evt[ret_type])) / len(returns[ret_type]))
    for ret_type in return_type
}

tau_q = {
    ret_type: {
        '95': tau_q_95,
        '97.5': tau_q_975,
        '99': tau_q_99,
        'evt': 1.0 / (1.0 - Q[ret_type])
    }
    for ret_type in return_type
}

## 4. Calcolo degli intervalli di ricorrenza e plot della loro distribuzione

Ora che abbiamo i threshold possiamo calcolare gli intervalli di ricorrenza e vederne la distribuzione.

In [ ]:
def get_recurrence_intervals(is_extreme: pd.DataFrame):
    """Get the recurrence intervals durations between extremes.
    
    Parameters
    ----------
    is_extreme: pd.DataFrame
        a DataFrame with the date on the index and 1 if the return at time t is extreme,
        0 otherwise. Must contain a single column named 'extreme'
    """
    assert isinstance(is_extreme.index, pd.DatetimeIndex)
    assert len(is_extreme.columns) == 1
    
    # convert to int
    data = is_extreme.astype(np.int8)
    data.loc[:, 'date'] = data.index
    data.index = pd.RangeIndex(len(is_extreme))
    
    data_is_extreme = data[data[data.columns[0]] == 1]
    
    intervals = []
    for i in range(1, len(data_is_extreme)):
        last_time = data_is_extreme.date.iloc[i - 1]
        current_time = data_is_extreme.date.iloc[i]
        
        n_days = data_is_extreme.index[i] - data_is_extreme.index[i - 1]
        
        intervals.append((last_time, current_time, n_days))
        
    return intervals

I quantili vanno presi al 95%, 97.5%, 99%. Creo quindi il `dict` che contiene gli intervalli di ricorrenza, organizzati secondo il tipo di return ed il tipo di threshold (quantile o evt):

In [ ]:
# quantili: attenzione che sono TUTTI POSITIVI
thresholds = {
    ret_type: {
        q_type: returns[ret_type].abs().quantile(float(q_type) / 100)
        for q_type in quantile_type[:-1]
    }
    for ret_type in return_type
}

for ret_type in return_type:
    thresholds[ret_type]['evt'] = abs(threshold_evt[ret_type])

extremes = {
    ret_type: {
        q_type: (returns[ret_type].abs() >= thresholds[ret_type][q_type]).astype(np.int8)
        for q_type in quantile_type
    }
    for ret_type in return_type
}

tmp_rec_int = {
    ret_type: {
        q_type: get_recurrence_intervals(pd.DataFrame(extremes[ret_type][q_type]))
        for q_type in quantile_type
    }
    for ret_type in return_type
}

recurrence_intervals = {
    ret_type: {
        q_type: pd.DataFrame(data={
            'last_extreme': [x[0] for x in tmp_rec_int[ret_type][q_type]],
            'current_extreme': [x[1] for x in tmp_rec_int[ret_type][q_type]],
            'n_days': [x[2] for x in tmp_rec_int[ret_type][q_type]],
        })
        for q_type in quantile_type
    }
    for ret_type in return_type
}

### 4.1 Plot istogrammi intervalli di ricorrenza

Ora visualizzo graficamente la lunghezza degli intervalli di ricorrenza con degli istogrammi, rispettivamente per i returns positivi, negativi ed assoluti.

In [ ]:
hist_labels = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $r$',
    'neg': r'Negative $-r$',
    'abs': r'Absolute $|r|$',
}

y_lims = {
    'pos': [0.0, 0.15],
    'neg': [0.0, 0.15],
    'abs': [0.0, 0.08],
}

fig, ax = pl.subplots(nrows=1, ncols=3, figsize=(18, 7))

# positive returns
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        curr = recurrence_intervals[ret_type][q_type].n_days
        sns.distplot(curr, kde=False, norm_hist=True, label=hist_labels[q_type], ax=ax[i])

    ax[i].legend()
    ax[i].set(title=titles[ret_type], ylim=y_lims[ret_type])

sns.despine()

### 4.2 Creazione delle tabelle come nel paper

Ora creo le tabelle riassuntive come a pagina 9 del paper di [Jiang et al](https://doi.org/10.1080/14697688.2017.1373843).

Prima mi creo due funzioncine e poi le chiamo.

In [ ]:
def get_single_table(intervals: pd.DataFrame, returns: pd.DataFrame, ret_type: str, thresh: float, col_name='perc'):
    """Get a single panel sub-table."""
    obsv = int(intervals.shape[0])
    mean = intervals['n_days'].mean()
    median = intervals['n_days'].median()
    std_dev = intervals['n_days'].std()
    skewness = intervals['n_days'].skew()
    kurtosis = intervals['n_days'].kurt()
    
    ret_mean = returns.mean()
    ret_std_dev = returns.std()
    
    m = (thresh - ret_mean) / ret_std_dev
#     print(f"\nRet_type: {ret_type}, q_type: {col_name}")
#     print(f"Threshold: {thresh:5.4f}, Mean: {ret_mean:5.4f}, m: {m:5.4f}")
        
    acf, qstat, pvals = stattools.acf(intervals['n_days'].values, qstat=True, nlags=30)
    rho1 = acf[1]
    _, p_rho1 = scipy.stats.pearsonr(
        intervals['n_days'].values[1:],
        intervals['n_days'].shift(periods=1).values[1:],
    )
    
    rho5 = acf[5]
    _, p_rho5 = scipy.stats.pearsonr(
        intervals['n_days'].values[5:],
        intervals['n_days'].shift(periods=5).values[5:],
    )

    Q30 = qstat[-1]
    p_Q30 = pvals[-1]
    
    index = pd.Index(data=[
        'm',
        'obsv',
        'mean',
        'median',
        'stdev',
        'skew',
        'kurt',
        'rho(1)',
        'p-value(rho1)',
        'rho(5)',
        'p-value(rho5)',
        'Q(30)',
        'p-value(Q30)',
    ])
    
    result = pd.DataFrame(data=[
        [m],
        [obsv],
        [mean],
        [median],
        [std_dev],
        [skewness],
        [kurtosis],
        [rho1],
        [p_rho1],
        [rho5],
        [p_rho5],
        [Q30],
        [p_Q30],
    ],
    index=index,
    columns=[col_name])
    
    return result

Il titolo è la sezione della tabella (Negative/Positive/Absolute), i quantili sono quelli che mi interessano e finiranno sulle colonne della tabella ed il risultato è un `dict` che ha come chiavi i titoli.

In [ ]:
tables = {
    ret_type: {
        q_type: get_single_table(recurrence_intervals[ret_type][q_type],
                                 returns[ret_type],
                                 ret_type,
                                 thresh=thresholds[ret_type][q_type],
                                 col_name=q_type)
        for q_type in quantile_type
    }
    for ret_type in return_type
}

panels = {
    ret_type: pd.concat([tables[ret_type][q_type] for q_type in quantile_type], axis='columns')
    for ret_type in return_type
}

Visualizziamo le tabelle:

In [ ]:
panels['pos']

In [ ]:
panels['neg']

In [ ]:
panels['abs']

### 4.3 Plot degli autocorrelogrammi

Vediamo con gli [autocorrelogammi](https://en.wikipedia.org/wiki/Correlogram) se c'è autocorrelazione nelle serie dei *recurrence interval*.

In [ ]:
# sulle righe il tipo di threshold, sulle colonne il tipo di return
fig, ax = pl.subplots(nrows=3, ncols=3, figsize=(27, 24))
fig.suptitle("Autocorrelation plots", fontsize=16)

tsaplots.plot_acf(recurrence_intervals['neg']['95']['n_days'].values, lags=30, ax=ax[0][0], title=r'$-r$, 95%')
tsaplots.plot_acf(recurrence_intervals['pos']['95']['n_days'].values, lags=30, ax=ax[0][1], title=r'$r$, 95%')
tsaplots.plot_acf(recurrence_intervals['abs']['95']['n_days'].values, lags=30, ax=ax[0][2], title=r'$|r|$, 95%')
ax[0, 0].set_ylabel("Pearson $R$")

tsaplots.plot_acf(recurrence_intervals['neg']['97.5']['n_days'].values, lags=30, ax=ax[1][0], title=r'$-r$, 97.5%')
tsaplots.plot_acf(recurrence_intervals['pos']['97.5']['n_days'].values, lags=30, ax=ax[1][1], title=r'$r$, 97.5%')
tsaplots.plot_acf(recurrence_intervals['abs']['97.5']['n_days'].values, lags=30, ax=ax[1][2], title=r'$|r|$, 97.5%')
ax[1, 0].set_ylabel("Pearson $R$")

# WARNING: non ci sono abbastanza dati
# tsaplots.plot_acf(recurrence_intervals['neg']['99']['n_days'].values, lags=30, ax=ax[2][0], title=r'$-r$, 99%')
# tsaplots.plot_acf(recurrence_intervals['pos']['99']['n_days'].values, lags=30, ax=ax[2][1], title=r'$r$, 99%')
# tsaplots.plot_acf(recurrence_intervals['abs']['99']['n_days'].values, lags=30, ax=ax[2][2], title=r'$|r|$, 99%')
# ax[2, 0].set_ylabel("Pearson $R$")

tsaplots.plot_acf(recurrence_intervals['neg']['evt']['n_days'].values, lags=30, ax=ax[2][0], title=r'$-r$, EVT')
tsaplots.plot_acf(recurrence_intervals['pos']['evt']['n_days'].values, lags=30, ax=ax[2][1], title=r'$r$, EVT')
tsaplots.plot_acf(recurrence_intervals['abs']['evt']['n_days'].values, lags=30, ax=ax[2][2], title=r'$|r|$, EVT')
ax[2, 0].set_ylabel("Pearson $R$")

for a in ax[2]:
    a.set_xlabel('lag in the recurrence interval array')

sns.despine()

Per interpretare i plot, bisogna ricordare che:

- sulle $x$ c'è il lag della serie temporale relativa ai giorni tra i movimenti estremi, cioè quella degli intervalli di ricorrenza. Vuol dire che $x=22$ significa il 22° intervallo di ricorrenza visto nel passato, prima di quello attuale, non 22 giorni prima di oggi. La distanza in giorni potrebbe anche essere un anno o più.
- sulle y c'è la correlazione di Pearson $R$

Apparentemente c'è autocorrelazione nei recurrence intervals selezionati con il quantile $q_{0.95}$ fino a 6 per positivi e negativi, 2 per gli assoluti.

Per i recurrence intervals con $q_{0.975}$ solo a 1 giorno per positivi e negativi, nessuna per gli assoluti.

Per i recurrence intervals con $q_{0.99}$ non c'è autocorrelazione.

### 4.4 Verifica relazione empirica

Verifichiamo ora la relazione empirica $\tau_Q = \frac{Q}{1 - Q}$ dove $Q$ è il quantile scelto (0.95, 0.975, 0.99), $\tau_Q$ l'intervallo di ricorrenza medio

In [ ]:
title_format = "{:>15}"*5
row_format = "{:>15.3f}{:>15}{:>15.3f}{:>15.3f}" + "{:>14.3f}%"
print(title_format.format('Quantile', 'Return type', 'tau_q', 'True mean', 'Error %'))
print(title_format.format('-'*15, '-'*15, '-'*15, '-'*15, '-'*15))

for q_type in quantile_type[:-1]:
    for i, name in enumerate(return_type):
        data_mean = recurrence_intervals[name][q_type]['n_days'].mean()
        
        q = float(q_type) / 100.0
        tau = 1.0 / (1.0 - q)
        
        perc_diff = (tau - data_mean) / data_mean
        
        print(row_format.format(q, name, tau, data_mean, perc_diff * 100))
        
        if i == len(return_type) - 1:
            print("")

In effetti, la relazione è valida con un margine di errore massimo di circa il $5\%$.

# 5. Determinazione della probabilità di pericolo - TODO

Gli autori definiscono la *hazard probability* come

\begin{equation}
    W(\Delta t | t) = \frac{\int_t^{t + \Delta t} p(\tau)d\tau}{\int_t^{\infty}p(\tau)d\tau}
\end{equation}

dove $p(\tau)$ è la distribuzione di probabilità (`pdf` per scipy).

La hazard probability definisce la probabilità che, dato che si è verificato un evento estremo $t$ giorni nel passato, ci sia un tempo di attesa $\Delta t$ ulteriore prima di un altro evento estremo.
Se consideriamo $W(1 | t)$ è simile al problema che abbiamo affrontato con la rete neurale.

Ora, nota la ditribuzione $p(\tau)$, si può derivare analiticamente l'integrale. Il problema è quindi: come trovare $p(\tau)$, e che forma ha?

Gli autori utilizzano una [stretched exponential distribution](https://en.wikipedia.org/wiki/Stretched_exponential_function), una [*q*-exponential distribution](https://en.wikipedia.org/wiki/Q-exponential_distribution) ed una [Weibull distribution](https://it.wikipedia.org/wiki/Distribuzione_di_Weibull). I parametri delle 3 distribuzioni vengono stimati tramite MLE.

Il flusso è il seguente:

1. scegli una distribuzione (s-exp, q-exp, Weibull)
2. riformula la parametrizzazione in funzione solo  dello *shape parameter*
3. calcola la log-likelihood utilizzando una semplice ricerca a griglia sui parametri liberi
4. i parametri che forniscono la massima log-likelihood sono quelli cercati, e trova la formula teorica della *hazard probability* con le equazioni del paper

Cominciamo con la Weibull, ma prima creiamo una funzione che calcoli la *hazard probability* empirica, con la formula

$$
W_{emp}(\Delta t | t) = \frac{\#(t < \tau \leq t + \Delta t)}{\#(\tau > t)}
$$

dove al numeratore c'è il numero di recurrence intervals con valore compreso in $(t, t + \Delta t]$, al denominatore il numero di recurrence intervals con valore maggiore di $t$, cioè nel range $(t, +\infty)$.

In [ ]:
def get_empirical_hazard_prob(rec_ints: np.ndarray, t, delta_t):
    """Compute the empirical hazard probability."""
    assert isinstance(rec_ints, np.ndarray)
    num = np.sum(np.logical_and(rec_ints > t, rec_ints <= t + delta_t))
    denom = np.sum(rec_ints > t)
    
    return num / denom

## 5.1 Fitting della Weibull

In `scipy.stats` è definita come

\begin{eqnarray}
&f(x, c) = c x^{c - 1} e^{-x^{c}} \\
&f(x, c, loc, scale) = \frac{1}{scale}f\left(\frac{x - loc}{scale}, c\right)
\end{eqnarray}

dove $c$ è lo *shape parameter*. Nel paper invece è

\begin{equation}
f(x, \beta, \alpha) = \frac{\alpha}{\beta} \left( \frac{\tau}{\beta} \right)^{\alpha - 1} e^{-\left( \frac{\tau}{\beta} \right)^{\alpha}}
\end{equation}

quindi la corrispondenza è

\begin{eqnarray}
&loc = 0 \\
&\beta = scale \\
&shape = c = \alpha
\end{eqnarray}

Ora dobbiamo stimare i parametri della Weibull con una maximum log-likelihood estimation (*MLE*). Riscrivendoli in funzione di $\tau_Q$ e $\beta = scale$ abbiamo

\begin{eqnarray}
&\beta = \frac{\tau_Q}{\Gamma \left( 1 + \frac{1}{\alpha} \right)} \\
cioè \\
&\beta = scale = \frac{\tau_Q}{\Gamma \left( 1 + \frac{1}{c} \right)}
\end{eqnarray}

Ricordiamo che $\tau_Q = \frac{1}{1 - Q}$ dove $Q$ è il quantile.

A questo punto la MLE ha formula:

$$ ln(L_w) = n \cdot ln\left( \frac{c}{\beta} \right) + \sum_{i=1}^{n} \left[ (c - 1) ln\left( \frac{\tau_i}{\beta} \right) - \left( \frac{\tau_i}{\beta} \right)^c \right] $$

dove $n$ è il numero di recurrence intervals, $t_i$ il corrispondente valore dell'intervallo di ricorrenza (es: 14 giorni, 4 giorni...).

Il flow è quindi, in questo caso:

1. a seconda del percentile (95% o EVT) calcolare $Q$ e quindi $\tau_Q$
2. utilizzare una ricerca con step $1e-6$ sul parametro $c = \alpha$, il quale risulta in un certo valore di $\beta$
3. utilizzare quei valori di $c$ e di $\beta$ nella MLE
4. trovare il massimo della MLE ed i corrispondenti valori di $c$ e $\beta$
5. urrà! Ora possiamo usarli nella *pdf* della distribuzione Weibull per ottenere l'hazard $W(\Delta t | t)$

In [ ]:
@timeit
@numba.jit(nopython=True, parallel=True, nogil=True)
def mle_weibull(rec_ints: np.ndarray, c: np.ndarray, beta: np.ndarray):
    """MLE estimation for weibull distribution, given an array of c shape parameters and the tau_q,
    with the recurrence intervals rec_ints.
    """       
    m = beta.shape[0]
    n = rec_ints.shape[0]

    # log-likelihood    
    log_likelihoods = np.zeros_like(beta)
    
    # precompute matrices for tau_beta and ln_tau_beta
    tau_beta = np.zeros((n, m), dtype=np.float64)
    for i in range(n):
        for j in range(m):
            tau_beta[i, j] = rec_ints[i] / beta[j]
            
    ln_tau_beta = np.log(tau_beta)
    
    c_beta = c / beta
    n_ln_c_beta = n * np.log(c_beta)
    c_1 = c - 1.0

    for j in numba.prange(m):  # no progress indication, it's a parallel for loop
        summ = 0
        
        for i in range(n):
            summ += c_1[j] * ln_tau_beta[i, j] - tau_beta[i, j] ** c[j]
            
        log_likelihoods[j] = n_ln_c_beta[j] + summ
        
    return log_likelihoods

Ora usiamo la funzione per calcolarci il fitting della Weibull per i returns positivi, negativi ed assoluti:

In [ ]:
c = np.arange(0.25, 2, 1e-3)
sfg = sfun.gamma(1.0 + (1.0 / c))

beta = {
    ret_type: {
        q_type: tau_q[ret_type][q_type] / sfg
        for q_type in quantile_type
    }
    for ret_type in return_type
}

i_ok = {
    ret_type: {
        q_type: np.argwhere(beta[ret_type][q_type] > 1e-6).flatten()
        for q_type in quantile_type
    }
    for ret_type in return_type
}

beta_ok = {
    ret_type: {
        q_type: beta[ret_type][q_type][i_ok[ret_type][q_type]]
        for q_type in quantile_type
    }
    for ret_type in return_type
}

c_ok = {
    ret_type: {
        q_type: c[i_ok[ret_type][q_type]]
        for q_type in quantile_type
    }
    for ret_type in return_type
}

In [ ]:
load_log_like = True

ll_weib_file = f"./log-like-weib_{split_key}.pickle"

if load_log_like:
    with open(ll_weib_file, 'rb') as infile:
        log_like_weib = pickle.load(infile)
else:
    log_like_weib = dict()

    for ret_type in return_type:
        log_like_weib[ret_type] = dict()
        print(f"\nReturn type: {ret_type}")

        for q_type in quantile_type:
            x = recurrence_intervals[ret_type][q_type]['n_days'].values
            print(f"Computing Weibull MLE on quantile: {q_type}, c={c_ok[ret_type][q_type].shape}, beta={beta_ok[ret_type][q_type].shape}")

            ll = mle_weibull(x, c_ok[ret_type][q_type], beta_ok[ret_type][q_type])

            log_like_weib[ret_type][q_type] = ll
          
    with open(ll_weib_file, 'wb') as outfile:
          pickle.dump(log_like_weib, outfile)

In [ ]:
colors_mle = {
    'evt': 'lightskyblue',
    '95': 'palegreen',
    '97.5': 'limegreen',
    '99': 'darkgreen',
}

legend_labels_mle = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $log(r)$',
    'neg': r'Negative $log(r)$',
    'abs': r'Absolute $log(r)$',
}

fig, ax = pl.subplots(nrows=3, ncols=1, figsize=(14, 15))

# positive log-returns
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        ax[i].plot(
            c_ok[ret_type][q_type],
            log_like_weib[ret_type][q_type],
            color=colors_mle[q_type],
            label=legend_labels_mle[q_type])

        i_max = np.argmax(log_like_weib[ret_type][q_type])
        
        ax[i].plot(
            c_ok[ret_type][q_type][i_max],
            log_like_weib[ret_type][q_type][i_max],
            marker='o',
            color=colors_mle[q_type]
        )
    
    ax[i].set(title=titles[ret_type])
    
for a in ax:
    a.set(xlabel=r'$c = \alpha$', ylabel=r'$log(L_W)$')
    a.legend(loc='lower right')

sns.despine()

Ok, ora che abbiamo le MLE per i tre tipi di returns e i minimi, possiamo fittare la Weibull sui recurrence intervals:

In [ ]:
i_min = {
    ret_type: {
        q_type: np.argmax(log_like_weib[ret_type][q_type])
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_shape = {
    'weibull': {
        ret_type: {
            q_type: c_ok[ret_type][q_type][i_min[ret_type][q_type]]
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
}

best_scale = {
    'weibull': {
        ret_type: {
            q_type: tau_q[ret_type][q_type] / sfun.gamma(1.0 + 1.0 / best_shape['weibull'][ret_type][q_type])
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
}

best_params = {
    'weibull': {
        ret_type: {
            q_type: {
                'shape': best_shape['weibull'][ret_type][q_type],
                'scale': best_scale['weibull'][ret_type][q_type],
                'loc': 0.0,
            }
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
}

## 5.2 Fitting della stretched-exponential (s-exp)

Creiamo ora la classe per la s-exp, che ha *pdf*:

$$
p(x, c, a, b) = a e^{-\left( bx \right)^c}
$$

dove $c$, $a$ e $b$ sono *shape parameters*, con $0 < c < 1$, $b \geq 0$ e $a > 0$.

Creo allora la funzione che minimizza la log-likelihood della s-exp, che è

$$
ln(L_{s-exp}) = n \cdot \ln(a) - \sum_{i=1}^{n} (b \cdot x_i)^c
$$

dove $n$ è il numero di recurrence intervals, $a = \frac{c \Gamma \left( \frac{2}{c} \right)}{\left[ \Gamma \left( \frac{1}{c} \right) \right]^2 \tau_Q}$ e $b = \frac{ \Gamma \left( \frac{2}{c} \right)}{\Gamma \left( \frac{1}{c} \right) \tau_Q}$.

In [ ]:
def get_a_b_sexp(c, tau_q):
    """Get the a and b params."""
    gamma_2_c = sfun.gamma(2.0 / c)
    gamma_1_c = sfun.gamma(1.0 / c)
    
    b_all = gamma_2_c / (gamma_1_c * tau_q)
    a_all = b_all * c / gamma_1_c
    
    return a_all, b_all

@timeit
def mle_sexp(rec_ints: np.ndarray, c: np.ndarray, tau_q: float):
    """MLE estimation for s-exponential distribution, given an array of c shape parameters and the tau_q,
    with the recurrence intervals rec_ints.
    """
    n = rec_ints.shape[0]
    
    a_all, b_all = get_a_b_sexp(c, tau_q)
    
    ln_a_all = np.log(a_all)
    
    ll = np.zeros((c.shape[0], ), dtype=np.float64)
    
    for j, c in enumerate(c):
        ssum = 0
        a = a_all[j]
        b = b_all[j]
        
        for i in range(n):
            ssum += np.power((b * rec_ints[i]), c)
            
        ll[j] = n * ln_a_all[j] - ssum
    
    ll[np.isnan(ll)] = -np.inf
        
    return ll

Ora usiamo la funzione per calcolarci il fitting della s-exp per i returns positivi, negativi ed assoluti:

In [ ]:
c_sexp = np.arange(1e-3, 1.0, 1e-3)

In [ ]:
load_log_like = True

ll_sexp_file = f"./log-like-sexp_{split_key}.pickle"

if load_log_like:
    with open(ll_sexp_file, 'rb') as infile:
        log_like_sexp = pickle.load(infile)
else:
    log_like_sexp = dict()

    for ret_type in return_type:
        log_like_sexp[ret_type] = dict()
        print(f"\nReturn type: {ret_type}")

        for q_type in quantile_type:
            x = recurrence_intervals[ret_type][q_type]['n_days'].values
            print(f"Computing s-exp MLE on quantile: {q_type}, c={c_sexp.shape}")

            ll = mle_sexp(x, c_sexp, tau_q[ret_type][q_type])

            log_like_sexp[ret_type][q_type] = ll
            
    log_like_sexp['c'] = c_sexp

    with open(ll_sexp_file, 'wb') as outfile:
          pickle.dump(log_like_sexp, outfile)

Ora prendiamo la massima log-likelihoood:

In [ ]:
i_max_sexp = {
    ret_type: {
        q_type: np.argmax(log_like_sexp[ret_type][q_type])
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_shape['s-exp'] = {
    ret_type: {
        q_type: c_sexp[i_max_sexp[ret_type][q_type]]
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_a_sexp = {
    ret_type: {
        q_type: get_a_b_sexp(best_shape['s-exp'][ret_type][q_type], tau_q[ret_type][q_type])[0]
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_b_sexp = {
    ret_type: {
        q_type: get_a_b_sexp(best_shape['s-exp'][ret_type][q_type], tau_q[ret_type][q_type])[1]
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_params['s-exp'] = {
    ret_type: {
        q_type: {
            'shape': best_shape['s-exp'][ret_type][q_type],
            'a': best_a_sexp[ret_type][q_type],
            'b': best_b_sexp[ret_type][q_type],
            'loc': 0.0,
        }
        for q_type in quantile_type
    }
    for ret_type in return_type
}

Plottiamo quindi i risultati della MLE:

In [ ]:
colors_mle = {
    'evt': 'lightskyblue',
    '95': 'palegreen',
    '97.5': 'limegreen',
    '99': 'darkgreen',
}

legend_labels_mle = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $log(r)$',
    'neg': r'Negative $log(r)$',
    'abs': r'Absolute $log(r)$',
}

fig, ax = pl.subplots(nrows=3, ncols=1, figsize=(14, 15))

# positive log-returns
truncation = 100
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        ax[i].plot(
            c_sexp[truncation:],
            log_like_sexp[ret_type][q_type][truncation:],
            color=colors_mle[q_type],
            label=legend_labels_mle[q_type])
        
        ax[i].plot(
            c_sexp[i_max_sexp[ret_type][q_type]],
            log_like_sexp[ret_type][q_type][i_max_sexp[ret_type][q_type]],
            marker='o',
            color=colors_mle[q_type]
        )
    
    ax[i].set(title=titles[ret_type])
    
for a in ax:
    a.set(xlabel=r'$c = \alpha$', ylabel=r'$log(L_{s-exp})$')
    a.legend(loc='lower right')

sns.despine()

## 5.3 Fitting della q-exponential

La terza distribuzione è la [q-exponential](https://en.wikipedia.org/wiki/Q-exponential_distribution).

Creo allora la funzione che minimizza la log-likelihood della q-exp, che è

$$
ln(L_{q-exp}) = n \cdot \ln[\lambda (2 - q)] - \frac{1}{q - 1} \sum_{i=1}^{n} \ln[1 + (q - 1) \lambda \tau_i]
$$

dove $n$ è il numero di recurrence intervals, $\tau_i$ il valore dell'i-esimo recurrence interval, e il parametro $\lambda$ si stima così:

$$
\lambda = \frac{1}{\tau_Q(3 - 2q)}
$$

il parametro libero $q$ ha il range $\left( 0, \frac{3}{2} \right)$.

In [ ]:
@timeit
def mle_qexp(rec_ints: np.ndarray, q: np.ndarray, tau_q: float):
    """MLE estimation for q-exponential distribution, given an array of q shape parameters and the tau_q,
    with the recurrence intervals rec_ints.
    """
    assert np.all(q < 1.5)
#     ipdb.set_trace()
    
    n = rec_ints.shape[0]
    m = q.shape[0]
    
    lam = 1.0 / (tau_q * (3 - 2 * q))
    
    ll = np.zeros((q.shape[0], ), dtype=np.float64)
    
    for j in range(m):
        ssum = 0
        
        for i in range(n):
            ssum += np.log(1 + (q[j] - 1) * lam[j] * rec_ints[i])
            
#         ipdb.set_trace()
        ll[j] = n * np.log(lam[j] * (2 - q[j])) - (1 / (q[j] - 1)) * ssum
        
    return ll

Ora usiamo la funzione per calcolarci il fitting della s-exp per i returns positivi, negativi ed assoluti:

In [ ]:
q_qexp = np.arange(1.0, 1.5, 1e-3)

In [ ]:
load_log_like = False

ll_qexp_file = f"./log-like-qexp_{split_key}.pickle"

if load_log_like:
    with open(ll_qexp_file, 'rb') as infile:
        log_like_qexp = pickle.load(infile)
else:
    log_like_qexp = dict()

    for ret_type in return_type:
        log_like_qexp[ret_type] = dict()
        print(f"\nReturn type: {ret_type}")

        for q_type in quantile_type:
            x = recurrence_intervals[ret_type][q_type]['n_days'].values
            print(f"Computing q-exp MLE on quantile: {q_type}, c={q_qexp.shape}")

            ll = mle_qexp(x, q_qexp, tau_q[ret_type][q_type])
            ll[np.isnan(ll)] = -np.inf
            
            log_like_qexp[ret_type][q_type] = ll
            
    log_like_qexp['c'] = q_qexp

    with open(ll_qexp_file, 'wb') as outfile:
          pickle.dump(log_like_qexp, outfile)

Ora prendiamo la massima log-likelihoood:

In [ ]:
i_max_qexp = {
    ret_type: {
        q_type: np.argmax(log_like_qexp[ret_type][q_type])
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_shape['q-exp'] = {
    ret_type: {
        q_type: q_qexp[i_max_qexp[ret_type][q_type]]
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_lambda_qexp = {
    ret_type: {
        q_type: 1.0 / (tau_q[ret_type][q_type] * (3 - 2 * best_shape['q-exp'][ret_type][q_type]))
        for q_type in quantile_type
    }
    for ret_type in return_type
}

best_params['q-exp'] = {
    ret_type: {
        q_type: {
            'shape': best_shape['q-exp'][ret_type][q_type],
            'lambda': best_lambda_qexp[ret_type][q_type],
            'loc': 0.0,
        }
        for q_type in quantile_type
    }
    for ret_type in return_type
}

Plottiamo quindi i risultati della MLE:

In [ ]:
colors_mle = {
    'evt': 'lightskyblue',
    '95': 'palegreen',
    '97.5': 'limegreen',
    '99': 'darkgreen',
}

legend_labels_mle = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $log(r)$',
    'neg': r'Negative $log(r)$',
    'abs': r'Absolute $log(r)$',
}

fig, ax = pl.subplots(nrows=3, ncols=1, figsize=(14, 15))

# positive log-returns
truncation = 100
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        ax[i].plot(
            q_qexp,
            log_like_qexp[ret_type][q_type],
            color=colors_mle[q_type],
            label=legend_labels_mle[q_type])
        
        ax[i].plot(
            q_qexp[i_max_qexp[ret_type][q_type]],
            log_like_qexp[ret_type][q_type][i_max_qexp[ret_type][q_type]],
            marker='o',
            color=colors_mle[q_type]
        )
    
    ax[i].set(title=titles[ret_type])
    
for a in ax:
    a.set_ylim([-1500, 0.0])
    a.set(xlabel=r'$c = q$', ylabel=r'$log(L_{q-exp})$')
    a.legend(loc='lower left')

sns.despine()

## 5.4 Calcolo Hazard Probability

Perfetto, ora ho i parametri della Weibull, della s-exp e della q-exp per ogni tipo di return e di threshold. Posso quindi ottenere la curva teorica per il fitting dei recurrence intervals.

Per la Weibull è

$$
W_W(\Delta t | t) = 1 - e^{\left[ \left( \frac{t}{\beta} \right)^\alpha - \left( \frac{t + \Delta t}{\beta} \right)^\alpha \right]}
$$

dove $\alpha = c^*$ lo *shape* ottimale, e $\beta = \frac{\tau_Q}{\Gamma \left( 1 + \frac{1}{\alpha} \right)}$ lo *scale* ottimale.

Per la s-exp è:

$$
W_{s-exp}(\Delta t | t) = \frac{\frac{bc}{a} - \Gamma_l \left( \frac{1}{c}, (bt)^c \right) - \Gamma_u \left( \frac{1}{c}, [b(t + \Delta t)]^c \right)}{\Gamma_u \left( \frac{1}{c}, (bt)^c \right)}
$$

Per la q-exp è:

$$
W_{q-exp}(\Delta t | t) = 1 - \left[ 1 + \frac{(q - 1)\lambda \Delta t}{1 + (q - 1)\lambda t} \right]^{1 - \frac{1}{q - 1}}
$$

Mi creo allora le funzioni che le calcolano:

In [ ]:
def weibull_hazard(t, shape, scale, delta_t=1):
    part_1 = np.power((t / scale), shape)
    part_2 = np.power(((t + delta_t) / scale), shape)
    
    hazard = 1 - np.exp(part_1 - part_2)
    
    return hazard

def sexp_hazard(t, c, a, b, delta_t=1):
    num1 = (b * c / a)
    num2 = sfun.gammainc((1.0 / c), np.power((b * t), c)) * sfun.gamma(1.0 / c)
    num3 = sfun.gammaincc((1.0 / c), np.power(b * (t + delta_t), c)) * sfun.gamma(1.0 / c)
    
    num = num1 - num2 - num3
    
    denom = sfun.gammaincc((1.0 / c), np.power(b * t, c))
    
    hazard = num / denom
    
    return hazard

def qexp_hazard(t, q, lam, delta_t=1):
    num = (q - 1) * lam * delta_t
    denom = 1 + (q - 1) * lam * t
    exponent = 1 - (1 / (q - 1))
    
    hazard = 1 - np.power((1 + num / denom), exponent)
    
    return hazard

In [ ]:
max_delta_t = 60
x = np.arange(max_delta_t + 1)

ret_type = 'neg'
q_type = '99'

theoretical_hazard = {
    'weibull': weibull_hazard(
        x,
        best_params['weibull'][ret_type][q_type]['shape'],
        best_params['weibull'][ret_type][q_type]['scale']
    ),
    's-exp': sexp_hazard(
        x,
        best_params['s-exp'][ret_type][q_type]['shape'],
        best_params['s-exp'][ret_type][q_type]['a'],
        best_params['s-exp'][ret_type][q_type]['b'],
    ),
    'q-exp': qexp_hazard(
        x,
        best_params['q-exp'][ret_type][q_type]['shape'],
        best_params['q-exp'][ret_type][q_type]['lambda'],
    )
}

empirical_hazard = np.array([
    get_empirical_hazard_prob(recurrence_intervals[ret_type][q_type]['n_days'].values, t, 1)
    for t in x
])

In [ ]:
fig, ax = pl.subplots(nrows=1, ncols=1, figsize=(16, 9))

dist_colors = {
    'weibull': 'orchid',
    's-exp': 'orangered',
    'q-exp': 'mediumblue'
}

dist_labels = {
    'weibull': r'$W_W$',
    's-exp': r'$W_{s-exp}$',
    'q-exp': r'$W_{q-exp}$',
}

for dist_type in distribution_type:
    ax.plot(
        x,
        theoretical_hazard[dist_type],
        color=dist_colors[dist_type],
        label=dist_labels[dist_type])
    
ax.plot(
    x,
    empirical_hazard_prob,
    label=r'$W_{emp}$',
    color='black',
    linestyle='-',
    marker='o',
    markersize=1,
    linewidth=0.5
)

ax.legend()
ax.set(xlabel=r'$t$', ylabel=r'$W(1 | \Delta t)$', title=r'Hazard probability for $q = 0.99$')

sns.despine()

## COSA NON FUNZIONA

- [ ] La EVT ha come shape parameter un numero maggiore di 1, non va bene perché la curva qui sopra viene al contrario.

In [ ]:
## NON servivano...sprecata mezza giornata, si cercano improperi creativi

# class QExponential(scipy.stats.rv_continuous):
#     """
#     The q-exponential distribution centered in 0 with scale 1 and shape q.
#     """
    
#     def __init__(self):
#         super().__init__()
        
#     def _get_support(self, *args):
#         print(f"args: {args}")
        
    
#     def _pdf(self, x, c, loc=0, scale=1, rate=1):
#         """Probability density function for the q-exponential distribution"""
#         exponent = - 1.0 / (c - 1)
#         if c == 1:
#             e_q = np.exp(c * x)
#         else:
#             e_q = np.power((1 + (c - 1) * rate * x), exponent)

#         pdf = (2 - c) * rate * e_q
        
#         return pdf
    
#     def _pdf(self, x, c):
#         return self._pdf(x, c, loc=0, scale=1, rate=1)

# class StretchedExponential(scipy.stats.rv_continuous):
#     """Stretched exponential distribution."""
    
#     def __init__(self):
#         """Create a new instance of the s-exp distribution."""
#         super().__init__()
        
#         self.a = 0.0
#         self.b = np.inf
    
#     def pdf(self, x, shapes, loc=0, scale=1):
#         c = shapes[0]
#         a = shapes[1]
#         b = shapes[2]
        
#         if loc != 0.0:
#             y = (x - loc) / scale
#             return self._pdf(y, c, a, b) / scale
#         else:
#             return self._pdf(x, c, a, b)
    
#     def _pdf(self, x, c, a, b):
#         """Probability density function."""
#         exponent = -np.power((b * x), c)
#         return a * np.exp(exponent)